In [22]:
import pandas as pd
import altair as alt
import eco_style
import pycountry
alt.themes.enable('light')

ThemeRegistry.enable('light')

In [49]:
df = pd.read_excel("data/migration_population.xlsx", skiprows=2)
df.columns = ['area'] + list(df.columns[1:])
df['country'] = df.area.shift(1)
df = df[~df.country.astype(str).str.contains('%')]
df = df.iloc[1:-1, :]
df = df.drop(columns=['area'])
df = df.melt(id_vars=['country'], var_name='date', value_name='value')
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df = df.dropna().sort_values(['country', 'date'])
full_df = df.copy()

recent_df = df.sort_values('date').drop_duplicates(subset=['country'], keep='last').rename(columns={'value': 'end'})
start_df = df.sort_values('date').drop_duplicates(subset=['country'], keep='first').rename(columns={'value': 'start'})

df = pd.merge(start_df, recent_df, on='country')

def get_iso3(country):
    try:
        return pycountry.countries.lookup(country).alpha_3
    except LookupError:
        manual = {'Türkiye': 'TUR'}
        if country in manual:
            return manual[country]
        print(f"Could not find ISO3 code for {country}")
        return None

df['iso3'] = df['country'].apply(get_iso3)

oecd_iso3s = ["AUS","AUT","BEL","CAN","CHL","COL","CZE","DNK","EST","FIN","FRA","DEU","GRC","HUN","ISL","IRL","ISR","ITA","JPN","KOR","LVA","LTU","LUX","MEX","NLD","NZL","NOR","POL","PRT","SVK","SVN","ESP","SWE","CHE","TUR","GBR","USA"]

df = df[df.iso3.isin(oecd_iso3s)]
df['series'] = '2022'

df['rank'] = df['end'].rank(ascending=False)

bars = alt.Chart(df).mark_bar(color="#36B7B4").encode(
    x=alt.X('end:Q', axis=alt.Axis(labelExpr='datum.value + "%"')),
    y=alt.Y('country:N', title='', 
            sort=alt.EncodingSortField(field='rank', order='ascending')),
    color=alt.Color('series:N', scale=alt.Scale(
        domain=['2022', '2012'],
        range=['#36B7B4', '#E6224B']
    ),
        legend=alt.Legend(orient='none', 
                          values=['2012', '2022'],
                          title='', direction='horizontal', legendX=0, legendY=-15)
    ),
    opacity=alt.condition(alt.datum.iso3 == 'GBR', alt.value(1), alt.value(0.5))
)

rules = alt.Chart(df).mark_circle(color='#E6224B').encode(
    x='start:Q',
    y=alt.Y('country:N', sort=alt.EncodingSortField(field='rank', order='ascending')),
    opacity=alt.condition(alt.datum.iso3 == 'GBR', alt.value(1), alt.value(0.5))
)

chart = (bars + rules).properties(
    width=400,
    height=400)

chart.save("oecd_migration_population.json")
chart.save("oecd_migration_population.png", scale_factor=3.0)

chart


/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

# LAD Map - Refugees

In [73]:
df = pd.read_excel("data/resettlement-LADs.xlsx", skiprows=1, sheet_name="Data - Res_D01")
def quarter_str_to_date(date):
    if pd.isna(date):
        return None
    date = str(date)
    quarter_num = int(date[-1])
    year = date[:4]
    month = (quarter_num-1)*3 + 1
    return f"{year}-{month:02d}-01"
df['date'] = df['Quarter'].apply(quarter_str_to_date)
df = df[['date', 'LAD Code', 'Local Authority', 'Resettlement Scheme', 'Persons']]
df = df.groupby(['Local Authority']).agg({'Persons': 'sum', 'LAD Code': 'last'}).reset_index()
df = df.rename(columns={'Persons': 'value', 'Local Authority': 'name', 'LAD Code': 'id'})
df = df[['id', 'name', 'value']]
df.to_csv("data/cumulative_resettlement-LADs.csv", index=False)

In [75]:
df.sort_values('value', ascending=False).head(10)

,id,name,value
313,Unknown,Unknown,20950.0
75,E08000026,Coventry,795.0
32,E08000032,Bradford,793.0
24,E08000025,Birmingham,657.0
66,S12000036,City of Edinburgh,621.0
124,S12000049,Glasgow City,524.0
122,E08000037,Gateshead,511.0
257,E08000019,Sheffield,483.0
37,E06000023,"Bristol, City of",425.0
23,N09000003,Belfast,419.0
